In [1]:
from sklearn.model_selection import KFold, train_test_split
import torch.distributions.distribution as D
import data.medain_filtering_class as mf
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import data.read_samples as rs
import torch.optim as optim
from RBM import new_RBM as RBM
import numpy as np
import datetime
import torch

print(datetime.datetime.now(), "model.py code start")

BATCH_SIZE = 10
EPOCH = 100
LEARNING_RATE = 0.2
ANNEALING_RATE = 0.999
VISIBLE_UNITS = [180, 200, 250]
HIDDEN_UNITS = [80, 100, 120]
K_FOLD = 1

def show_adn_save(result, file_name=None):
    nprst = np.array(result)
    f = ".txt" % file_name
    print(nprst)

print("[MODL] Model main code is starting....")

print("[INFO] Read train data, cross-vaildation data and test data from median filtering code")
dataset_db1, dataset_db2, dataset_db3 = mf.ecg_filtering(True)
db1_sig, db1_label, db2_sig, db2_label, db3_sig, db3_label = rs.return_list()

train_dataset = list(mf.list_to_list(dataset_db1)) * 4
cross_dataset = list(mf.list_to_list(dataset_db2)) * 4
test_dataset = list(mf.list_to_list(dataset_db3))  * 4

[INFO] Pre-processing for make clean
2022-05-11 16:08:06.523857 model.py code start
[MODL] Model main code is starting....
[INFO] Read train data, cross-vaildation data and test data from median filtering code
[INFO] Read file and indexing start...
[INFO]	final_db1 direcotry found.
......	...................i	Current_Index	From_Index
[IWIP]	final_db1 reading... 0 0 200
[IWIP]	final_db1 reading... 1 200 400
[IWIP]	final_db1 reading... 2 400 600
[IWIP]	final_db1 reading... 3 600 800
[IWIP]	final_db1 reading... 4 800 1000
[IWIP]	final_db1 reading... 5 1000 1200
[IWIP]	final_db1 reading... 6 1200 1400
[IWIP]	final_db1 reading... 7 1400 1600
[IWIP]	final_db1 reading... 8 1600 1800
[IWIP]	final_db1 reading... 9 1800 2000
[IWIP]	final_db1 reading... 10 2000 2200
[IWIP]	final_db1 reading... 11 2200 2400
[IWIP]	final_db1 reading... 12 2400 2600
[IWIP]	final_db1 reading... 13 2600 2800
[IWIP]	final_db1 reading... 14 2800 3000
[IWIP]	final_db1 reading... 15 3000 3200
[IWIP]	final_db1 reading... 1

/Users/bahk_insung/miniforge3/lib/python3.9/site-packages/scipy/signal/_signaltools.py:1531: UserWarning: kernel_size exceeds volume extent: the volume will be zero-padded.
  warnings.warn('kernel_size exceeds volume extent: the volume will be '


[INFO]	final_db3 direcotry found.
[DONE] Pre-processing is done.
[ERRR] Got a issue with create file
[INFO] Read records file from  ./data/db1/
[RSLT]			 Export records ...
		 ['100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '111', '112', '113', '114', '115', '116', '117', '118', '119', '121', '122', '123', '124']
[INFO]./rdsamp commending start
[IWIP]		rdsamp Converting 100
[IWIP]		rdsamp Converting 101
[IWIP]		rdsamp Converting 102
[IWIP]		rdsamp Converting 103
[IWIP]		rdsamp Converting 104
[IWIP]		rdsamp Converting 105
[IWIP]		rdsamp Converting 106
[IWIP]		rdsamp Converting 107
[IWIP]		rdsamp Converting 108
[IWIP]		rdsamp Converting 109
[IWIP]		rdsamp Converting 111
[IWIP]		rdsamp Converting 112
[IWIP]		rdsamp Converting 113
[IWIP]		rdsamp Converting 114
[IWIP]		rdsamp Converting 115
[IWIP]		rdsamp Converting 116
[IWIP]		rdsamp Converting 117
[IWIP]		rdsamp Converting 118
[IWIP]		rdsamp Converting 119
[IWIP]		rdsamp Converting 121
[IWIP]		rdsamp Converting 122


KeyboardInterrupt: 

In [ ]:
size_str = '''
{} {} {} {} {} {} 
{} {} {}
'''.format(len(db1_sig), len(db1_label), len(db2_sig), len(db2_label), len(db3_sig), len(db3_label),
            len(train_dataset), len(cross_dataset), len(test_dataset))
print(size_str)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    (db1_sig + db2_sig), 
    (db1_label + db2_label),
    test_size=0.33,
    shuffle=True
)

print()

train_dataloader = DataLoader(X_train,
                              batch_size=BATCH_SIZE,
                              num_workers=0, collate_fn=lambda x: x,
                              shuffle=True)

test_dataloader = DataLoader(X_test,
                             batch_size=BATCH_SIZE,
                             shuffle=True)

In [ ]:
rbm_first = RBM(n_vis=VISIBLE_UNITS[0], n_hid=HIDDEN_UNITS[0], k=K_FOLD, batch=BATCH_SIZE)
rbm_second = RBM(n_vis=VISIBLE_UNITS[1], n_hid=HIDDEN_UNITS[1], k=K_FOLD, batch=BATCH_SIZE)
rbm_third = RBM(n_vis=VISIBLE_UNITS[2], n_hid=HIDDEN_UNITS[2], k=K_FOLD, batch=BATCH_SIZE)

first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

gb_first_train_op = optim.SGD(rbm_first.parameters(), 0.1)
gb_second_train_op = optim.SGD(rbm_second.parameters(), 0.1)
gb_third_train_op = optim.SGD(rbm_third.parameters(), 0.1)

output_from_first = list()
output_from_second = list()
output_from_third = list()

In [ ]:
'''Train Part'''

loss_ = []
for epoch in range(EPOCH):
    '''First bbrbm'''
    for _, (data) in enumerate(train_dataloader):
        try:
            data = Variable(torch.tensor(data, dtype=torch.float32)).uniform_(0, 1)
            print("Success")
        except ValueError:
            print("Fail")
            pass

        data = torch.tensor(data)
        sample_data = torch.bernoulli(data)
        sample_data = torch.flatten(sample_data.clone())

        # tensor binary
        vog_first, v1, mt = rbm_first(sample_data)
        
        loss_first = rbm_first.free_energy(vog_first) - rbm_first.free_energy(v1)
        loss_.append(loss_first.data)
        
        first_train_op.zero_grad()
        loss_first.backward()
        first_train_op.step()
    
    output_from_first.append(v1.tolist())
    print("1ST BBrbm_first Training loss for {0} epoch {1}\tEstimate time : {2}".format(epoch, np.mean(loss_), mt))
